In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount = 'True')

Mounted at /content/drive/


In [ ]:
import numpy as np
import pandas as pd

import re
import string
import csv
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM, Dense, Input, Embedding, Concatenate, TimeDistributed, Bidirectional, GRU
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from keras.utils.vis_utils import plot_model
! pip install rouge
from rouge import Rouge
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#train.csv
train_df = pd.read_csv('/content/drive/MyDrive/Statistical Software - MSDS Fall 2022/Data/train.csv')

In [ ]:
#validation csv
val_df = pd.read_csv('/content/drive/MyDrive/Statistical Software - MSDS Fall 2022/Data/validation.csv')

In [ ]:
#test csv
test_df = pd.read_csv('/content/drive/MyDrive/Statistical Software - MSDS Fall 2022/Data/test.csv')

In [ ]:
print('train:',train_df.shape, '\nvalidation:', val_df.shape,'\ntest:', test_df.shape)

train: (287113, 3) 
validation: (13368, 3) 
test: (11490, 3)


In [ ]:
test_df.head()

,id,article,highlights
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,91b7d2311527f5c2b63a65ca98d21d9c92485149,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,caabf9cbdf96eb1410295a673e953d304391bfbb,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...
4,3da746a7d9afcaa659088c8366ef6347fe6b53ea,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6..."


## Checking Null Values

In [ ]:
train_df.isna().sum()

id            0
article       0
highlights    0
dtype: int64

## Checking Duplicates

In [ ]:
train_df.duplicated(subset=['article', 'highlights']).sum()

3098

In [ ]:
train_df = train_df.drop_duplicates(subset= ['article', 'highlights'])

In [ ]:
train_df = train_df.drop(['id'], axis = 1)
train_df = train_df.reset_index(drop=True)

In [ ]:
val_df = val_df.drop(['id'], axis = 1)
val_df = val_df.reset_index(drop=True)

In [ ]:
test_df = test_df.drop(['id'], axis = 1)
test_df = test_df.reset_index(drop=True)

# Applying Extractive Summarization
### To get smaller inputs for seq2seq model and only using the most important sentences.

In [ ]:
train_df.head()

,article,highlights
0,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...


In [ ]:
# Taking only 500 instances of testing data since we used the same for intermediate report.
small_df = pd.DataFrame()
small_df['text'] = test_df['article'][:500]
small_df['summary'] = test_df['highlights'][:500]

In [ ]:
small_df.head()

,text,summary
0,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...
4,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6..."


In [ ]:
small_df['text'][0]

"Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans. 'In a world where animals have more rights to space and food than humans,' said Charlie Leocha, consumer representative on the committee.\xa0'It is time that the DOT and FAA take a stand for humane treatment of passengers.' But could crowding on planes lead to more serious issues than fighting for sp

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

import nltk
nltk.download('punkt')

nltk_generated_summaries=[]

#text = origin_text
#original_summary = origin_summary
#original_summary = origin_summary

# Next, you need to tokenize the text:
for i in range(len(small_df)):
  stopWords = set(stopwords.words("english"))
  words = word_tokenize(small_df['text'][i])

  # Now we create a frequency table to keep a score of each word:

  freqTable = dict()
  for word in words:
      word = word.lower()
      if word in stopWords:
          continue
      if word in freqTable:
          freqTable[word] += 1
      else:
          freqTable[word] = 1

  # creating a dictionary to keep the score of each sentence:

  sentences = sent_tokenize(small_df['text'][i])
  sentenceValue = dict()

  for sentence in sentences:
      for word, freq in freqTable.items():
          if word in sentence.lower():
              if word in sentence.lower():
                  if sentence in sentenceValue:
                      sentenceValue[sentence] += freq
                  else:
                      sentenceValue[sentence] = freq

  sumValues = 0
  for sentence in sentenceValue:
      sumValues += sentenceValue[sentence]

  # defining the average value from the original text as such:

  average = int(sumValues / len(sentenceValue))

  # now storing the sentences into our summary:

  summary = ''

  for sentence in sentences:
      if (sentence in sentenceValue) and (sentenceValue[sentence] > (1.2 * average)):
          summary += " " + sentence
  print(i)
  nltk_generated_summaries.append(summary)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
df_extractive_generated = pd.DataFrame(columns=['article', 'summary_generated'])
df_extractive_generated['article']=small_df['text']
df_extractive_generated['summary_generated']=nltk_generated_summaries
df_extractive_generated.head()

,article,summary_generated
0,Ever noticed how plane seats appear to be gett...,"This week, a U.S consumer advisory group set ..."
1,A drunk teenage boy had to be rescued by secur...,"Rahul Kumar, 17, clambered over the enclosure..."
2,Dougie Freedman is on the verge of agreeing a ...,Freedman has stabilised Forest since he repla...
3,Liverpool target Neto is also wanted by PSG an...,Liverpool target Neto is also wanted by PSG a...
4,Bruce Jenner will break his silence in a two-h...,"The former Olympian and reality TV star, 65, ..."


In [ ]:
with open('/content/drive/MyDrive/Statistical Software - MSDS Fall 2022/Data/result_extractive_nltk.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['Article', 'Original Summary', 'Model Output'])
    for i in range(500):
        writer.writerow([df_extractive_generated.article[i], small_df.summary[i], df_extractive_generated.summary_generated[i]])

In [ ]:
rouge = Rouge()
rouge.get_scores(nltk_generated_summaries, small_df['summary'], avg=True, ignore_empty=True)

{'rouge-1': {'r': 0.56521753051062,
  'p': 0.18613200757035214,
  'f': 0.26560001490028456},
 'rouge-2': {'r': 0.24340476099815211,
  'p': 0.0672188943622949,
  'f': 0.09838800436679603},
 'rouge-l': {'r': 0.5263829792002735,
  'p': 0.17287314515526106,
  'f': 0.24681781492635868}}